In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
import pyspark
print(pyspark.__version__)

2.4.5


In [5]:
from google.colab import files
file = files.upload()

Saving cruise_ship_info.csv to cruise_ship_info.csv


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ship').getOrCreate()

In [0]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [8]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [9]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [17]:
df.head(5)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)]

In [51]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [18]:
# Cruise_line is important feature and needs to be converted into numerical value using StringIndexer
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_index').fit(df)

df_indexed = indexer.transform(df)
df_indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|       

In [19]:
# Drop Cruise_line since it has been replaced by its numerical equivalent Cruise_line_index
df_new = df_indexed.drop('Cruise_line')
df_new.show()

+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_index|
+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
|   Conquest| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|
|    Destiny| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|
|    Ecstasy| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|              1.0|
|    Elation| 15|            70.367|     20.52|  8.55|  10.2|            

In [20]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

df_new.columns

['Ship_name',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index']

In [0]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density',
                                       'Cruise_line_index'], 
                            outputCol='features')

output = assembler.transform(df_new)

In [22]:
output.columns

['Ship_name',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index',
 'features']

In [23]:
output.show()

+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+--------------------+
|  Ship_name|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_index|            features|
+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+--------------------+
|    Journey|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|[6.0,30.276999999...|
|      Quest|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|[6.0,30.276999999...|
|Celebration| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|[26.0,47.262,14.8...|
|   Conquest| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|[11.0,110.0,29.74...|
|    Destiny| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|[17.0,101.353,26....|
|    Ecs

In [25]:
output.head(1)[0]

Row(Ship_name='Journey', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_index=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))

In [27]:
# Now all i care about is the features & the labels (crew in this case)
final_data = output.select('features', 'crew')
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [0]:
# Now split this into training & test data
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [30]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               113|
|   mean| 7.630353982300896|
| stddev|3.6592565914607826|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [31]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                45|
|   mean| 8.205555555555557|
| stddev|3.0775496122989816|
|    min|              0.88|
|    max|              13.6|
+-------+------------------+



In [0]:
# Now time for linear regression
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features', labelCol='crew')

r = lr.fit(train_data)

In [39]:
test_results = r.evaluate(test_data)

# The residuals show the difference between the predicted value and the actua labels form the test data
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.9613593129885301|
|  -1.200298982504048|
|  0.5180267653340938|
| -0.5595215051612996|
| -0.2967640672045704|
|  -0.786756608219056|
|-0.23678719160944794|
|   0.433168350763518|
|-0.26329356746394694|
|  1.0597123414526735|
|  1.8397506412406668|
| 0.14141410196704562|
|-0.22830359251712373|
| -0.6024490884075764|
|  0.1825042913037631|
|-0.27384013386714656|
|  0.9491662428285323|
|-0.00611810673048...|
| 0.04176767133967818|
|  0.9174301237097175|
+--------------------+
only showing top 20 rows



In [42]:
test_results.meanSquaredError

0.8474762712865036

In [43]:
test_results.r2

0.90848816607882

In [46]:
# Let's deploy this model on data with only features & no labels
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[5.0,122.0,28.5,1...|
|[6.0,90.0,20.0,9....|
|[6.0,93.0,23.94,9...|
|[6.0,112.0,38.0,9...|
|[6.0,158.0,43.7,1...|
|[7.0,116.0,31.0,9...|
|[7.0,158.0,43.7,1...|
|[8.0,77.499,19.5,...|
|[9.0,59.058,17.0,...|
|[9.0,113.0,26.74,...|
|[10.0,46.0,7.0,6....|
|[10.0,77.0,20.16,...|
|[10.0,81.76899999...|
|[10.0,110.0,29.74...|
|[11.0,58.6,15.66,...|
|[11.0,90.09,25.01...|
|[11.0,108.977,26....|
|[12.0,25.0,3.88,5...|
|[12.0,42.0,14.8,7...|
|[12.0,88.5,21.24,...|
+--------------------+
only showing top 20 rows



In [52]:
predictions = r.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,122.0,28.5,1...|  5.73864068701147|
|[6.0,90.0,20.0,9....|10.200298982504048|
|[6.0,93.0,23.94,9...|10.571973234665906|
|[6.0,112.0,38.0,9...|  11.4595215051613|
|[6.0,158.0,43.7,1...| 13.89676406720457|
|[7.0,116.0,31.0,9...|12.786756608219056|
|[7.0,158.0,43.7,1...|13.836787191609448|
|[8.0,77.499,19.5,...| 8.566831649236482|
|[9.0,59.058,17.0,...| 7.663293567463947|
|[9.0,113.0,26.74,...|11.320287658547327|
|[10.0,46.0,7.0,6....| 2.630249358759333|
|[10.0,77.0,20.16,...| 8.858585898032954|
|[10.0,81.76899999...| 8.648303592517124|
|[10.0,110.0,29.74...|12.202449088407576|
|[11.0,58.6,15.66,...|7.4174957086962365|
|[11.0,90.09,25.01...| 8.753840133867147|
|[11.0,108.977,26....|11.050833757171468|
|[12.0,25.0,3.88,5...|2.8761181067304844|
|[12.0,42.0,14.8,7...| 6.758232328660322|
|[12.0,88.5,21.24,...| 9.372569876290282|
+--------------------+------------